# simaple rag with rerank 작성예정

### ElasticSearch에서 ReRank가 통합되어 있어서 <br> es에 rerank를 추가 <br>es의 kibana에서 dev tools 혹은 wget, curl을 통하 아래 커맨드를 수행

### Rerank를 지원하는 ES 버젼은 8.14 이기 때문에, 8.13 이하 버젼은 8.14로 버젼업을 해야함.

PUT _inference/rerank/cohere_rerank <br>
{<br>
    "service": "cohere",<br>
    "service_settings": {<br>
        "api_key": <API-KEY>, <br>
        "model_id": "rerank-english-v3.0"<br>
    },<br>
    "task_settings": {<br>
        "top_n": 10,<br>
        "return_documents": true<br>
    }<br>
}

### 영어가 아니기 때문에 모델을 "rerank-multilingual-v3.0"로 바꿈.

## vectorstore : es8

# 필요한 패키지 설치

In [1]:
# !openai migrate
# openai 1.x.x 이상 버젼을 사용할 경우 위 명령을 수행해야 langchain_community오류가 해결됨. 
# 오류 메세지는 open.error.ApiError과 같은 오류.
# openai migrate가 동작하지 않을 경우 
# 아래 명령을 수행. grit이 설치 되어 있을 경우 1라인은 스킵 가능.
# curl -fsSL https://docs.grit.io/install | bash
# grit install
# grit apply openai

# 위 명령을 수행하면 python 의 site_package 에서 langchain_community에서 python파일을 로딩해서 오류 구문을 수정. 
# 단 아래와 같은 코드는 아직도 버그임. 
# | retry_if_exception_type(openai.ServiceUnavailableError)
# openai 에는 ServiceUnavailableError 에러 클래스가 없음..

In [2]:
import openai
print(openai.__version__)
import langchain_community
print(langchain_community.__version__)

1.37.1
0.2.10


In [3]:
# %pip install -qU openai
# %pip install -qU langchain
# %pip install langchainhub
# %pip install -qU langchain-openai
# %pip install -U langchain_community
# %pip install langchain_elasticsearch
# %pip install -U langchain-cohere

In [4]:
import os
# from langchain_community.vectorstores.elastic_vector_search import ElasticKnnSearch
# from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy, ElasticsearchEmbeddings
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchRetriever

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
# embedding 패키지 로딩
from langchain_community.embeddings import HuggingFaceHubEmbeddings

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ES_HOST = os.getenv("ES_HOST")
ES_USER_ID = os.getenv("ES_ID")
ES_PASSWD = os.getenv("ES_PWD")
ES_INDEX_NAME = os.getenv("ES_INDEX_NAME")
EMBEDDING_BASE_URL=os.getenv("EMBEDDING_BASE_URL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGUNGFACE_TOKEN")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

### embedding 정의

In [7]:
embeddings = HuggingFaceHubEmbeddings(model=EMBEDDING_BASE_URL, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpointEmbeddings`.
  warn_deprecated(


In [8]:
embeddings.client

<InferenceClient(model='http://192.168.0.10:3000', timeout=None)>

In [9]:
embeddings.embed_documents("테스트")[0][0:3]

[-0.017445877, 0.036139887, -0.017976798]

### llm. 정의

In [10]:
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

In [11]:
llm.invoke("안녕")

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 9, 'total_tokens': 20}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None}, id='run-df27950c-a674-49bc-99c9-73e667ee6c18-0', usage_metadata={'input_tokens': 9, 'output_tokens': 11, 'total_tokens': 20})

### ES 모듈 로딩

In [12]:
es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)

/var/folders/jy/5_rl6575533638lskdr9s6s00000gn/T/ipykernel_7524/99351195.py:1: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://192.168.0.10:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [13]:
es.info()

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'name': 'es01', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'aZJrg1ffSsCl52WsqYpJvQ', 'version': {'number': '8.14.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd55f984299e0e88dee72ebd8255f7ff130859ad0', 'build_date': '2024-07-07T22:04:49.882652950Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Retriever 정의

In [19]:
import typing as t 
def hybrid_query(search_query: str) -> t.Dict:
    vector = embeddings.embed_query(search_query)  # same embeddings as for indexing
    return {
        "_source": ["text","metadata"], 
        "query": {
            "match": {
                'text': search_query,
            },
        },
        "knn": {
            "field": 'vector',
            "query_vector": vector,
            "k": 10,
            "num_candidates": 100,
        },
        "rank": {"rrf": {}},
        
    }
from typing import Any, Callable, Dict, List, Mapping, Optional, Sequence, Union, cast

es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, request_timeout=120)

knn_retriever = ElasticsearchRetriever(
    es_client=es,
    index_name = ES_INDEX_NAME,
    body_func=hybrid_query,
    content_field="text",
)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://192.168.0.10:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


### Cohere Reranker

In [20]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

In [21]:
# Cohere?

In [22]:
cohere_llm = Cohere(temperature=0, cohere_api_key=COHERE_API_KEY)
compressor = CohereRerank(model="rerank-multilingual-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=knn_retriever
)

compressed_docs = compression_retriever.invoke(
    "삼성전자", k=10, candidate=10
)
print(compressed_docs)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[Document(metadata={'_index': 'news_article_embedding', '_id': 'c6139d40-d308-46bc-b019-bf5627bde3a1', '_score': None, '_rank': 5, '_ignored': ['text.keyword'], '_source': {'metadata': {'id': 475372, 'hash_key': '3b2e7bbf13415c7f8f475cbd2678d7a2', 'title': '작년 말 삼성전자 소액주주 467만명… 1년 새 114만명 떠났다', 'created_date': '2024-03-13T06:57:07', 'portal': 'daum', 'media': '조선비즈', 'url': 'https://v.daum.net/v/20240313065707437', 'image_url': 'https://img1.daumcdn.net/thumb/S1200x630/?fname=https://t1.daumcdn.net/news/202403/13/chosunbiz/20240313065713108qiol.jpg'}}, 'relevance_score': 0.9993929}, page_content='작년 말 삼성전자 소액주주 467만명… 1년 새 114만명 떠났다\n삼성전자 주식을 보유한 소액주주 수가 지난해 114만명 넘게 줄었다.\n13일 금융감독원 전자공시시스템에 따르면 작년 말 기준 지분율 1% 미만 삼성전자 소액주주는 467만2039명으로 집계됐다.\n1년 전인 2022년 말 삼성전자 소액주주 수 581만3977명과 비교해 114만1938명 감소했다.\n한편 삼성전자 소액주주 467만2039명은 지난해 말 기준 삼성전자 주식 40억1789만2514주를 보유한 것으로 집계됐다.\n이 글자크기로 변경됩니다.\n(예시) 가장 빠른 뉴스가 있고 다양한 정보, 쌍방향 소통이 숨쉬는 다음뉴스를 만나보세요. 다음뉴스는 국내외 주요이슈와 실시간 속보, 문화생활 및 다양한 분야의 뉴스를 입체적으로 전

### rag prompt 로딩

In [23]:
prompt = hub.pull("rlm/rag-prompt")

In [28]:
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [24]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [26]:
rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
rag_chain.invoke("최근 삼성전자 관련 기사를 검색하고 각각의 기사에 대한 내용을 요약하고 날짜를 같이 표시해줘")

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'2024년 3월 28일, 삼성전자 주가가 장중 8만원을 넘었다는 기사가 두 번 보도되었습니다.'

In [ ]:
es.close()